---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

## Part 1 - Document Similarity

In [1]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    token = nltk.word_tokenize(doc)
    tags = nltk.pos_tag(token)
    synsets_list = []
    for token, wordnet_tag in tags:
        wordnet_tag = convert_tag(wordnet_tag)
        synset = wn.synsets(token, wordnet_tag)
        if len(synset) != 0:
            synsets_list.append(synset[0])

    return synsets_list


def similarity_score(s1, s2):
    max_sim_val = []

    for synset1 in s1:
        all_sim = []
        for synset2 in s2:
            sim_val = wn.path_similarity(synset1, synset2)
            if sim_val is not None:
                all_sim.append(sim_val)
        if len(all_sim) != 0:
            max_sim_val.append(max(all_sim))

    result = sum(max_sim_val)/len(max_sim_val)

    return result


def document_path_similarity(doc1, doc2):
    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [2]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

In [3]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head()

,Quality,D1,D2
0,1,"Ms Stewart, the chief executive, was not expec...","Ms Stewart, 61, its chief executive officer an..."
1,1,After more than two years' detention under the...,After more than two years in detention by the ...
2,1,"""It still remains to be seen whether the reven...","""It remains to be seen whether the revenue rec..."
3,0,"And it's going to be a wild ride,"" said Allan ...","Now the rest is just mechanical,"" said Allan H..."
4,1,The cards are issued by Mexico's consulates to...,The card is issued by Mexico's consulates to i...


In [4]:
def most_similar_docs():
    
    # Your Code Here
    df = paraphrases.copy()
    df['Similarity'] = df.apply(lambda row: document_path_similarity(row['D1'], row['D2']), axis=1)
    answer = df.loc[df['Similarity'] == df['Similarity'].max()].squeeze().values
    result = (answer[1], answer[2], answer[3])
    
    return result

most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.97530864197530864)

In [5]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    # Your Code Here
    def label(row):
        if row['Similarity'] > .75:
            row['Labels'] = 1
        else:
            row['Labels'] = 0
        return row

    df = paraphrases.copy()
    df['Similarity'] = df.apply(lambda row: document_path_similarity(row['D1'], row['D2']), axis=1)
    df = df.apply(label, axis=1)
    result = accuracy_score(df['Quality'], df['Labels'])

    return result

label_accuracy()

0.80000000000000004

## Part 2 - Topic Modelling

In [6]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [7]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=10, id2word=id_map, passes=25, random_state=34)

KeyboardInterrupt: 

### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

In [ ]:
def lda_topics():
    
    # Your Code Here
    
    return ldamodel.print_topics()

lda_topics()

### topic_distribution

In [ ]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [ ]:
def topic_distribution():
    
    # Your Code Here
    new_doc_vect = vect.transform(new_doc)
    corp = gensim.matutils.Sparse2Corpus(new_doc_vect, documents_columns=False)
    result =  list(ldamodel.get_document_topics(corp))[0]
    
    return result

### topic_names

In [ ]:
# incorrect
def topic_names():
    
    # Your Code Here
    labels = ['Health', 'Automobiles', 'Government', 'Travel', 'Computers & IT', 'Sports', 'Business', 'Society & Lifestyle', 'Region', 'Education']

    topics = lda_topics()

    res = []

    for _, x in topics:
        print(x)
        similarity = []
        for t in labels:
            similarity.append(document_path_similarity(x, t))
        best_match = sorted(zip(similarity, labels))[-1][1]
        res.append(best_match)
        
    return res

topic_names()